In [24]:
import csv
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

url_recetas = "https://medlineplus.gov/spanish/recetas/desayuno/"

In [25]:
response = requests.get(url_recetas)
enlaces_secciones = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    enlaces_secciones.append("https://medlineplus.gov/spanish/recetas/desayuno/")
    div_mp_content = soup.find('div', class_='mp-content')
    if div_mp_content:
        sections = div_mp_content.find_all('section')
        if len(sections) > 1:
            segunda_section = sections[1]
            links = segunda_section.find_all('a')
            for link in links:
                href = link.get('href')
                if href:
                    enlaces_secciones.append(href)
        else:
            print("No se encontró una segunda sección.")
    else:
        print("No se encontró el div con la clase mp-content.")

else:
    print(f"Error al acceder a la página: {response.status_code}")

print(enlaces_secciones)

['https://medlineplus.gov/spanish/recetas/desayuno/', 'https://medlineplus.gov/spanish/recetas/almuerzo/', 'https://medlineplus.gov/spanish/recetas/cena/', 'https://medlineplus.gov/spanish/recetas/postres/', 'https://medlineplus.gov/spanish/recetas/acompanamientos/', 'https://medlineplus.gov/spanish/recetas/bajo-en-grasa/', 'https://medlineplus.gov/spanish/recetas/bajo-en-sodio/', 'https://medlineplus.gov/spanish/recetas/bebidas/', 'https://medlineplus.gov/spanish/recetas/bocadillos/', 'https://medlineplus.gov/spanish/recetas/ensaladas/', 'https://medlineplus.gov/spanish/recetas/pan/', 'https://medlineplus.gov/spanish/recetas/salsas-y-aderezos/', 'https://medlineplus.gov/spanish/recetas/sin-gluten/', 'https://medlineplus.gov/spanish/recetas/sin-lacteos/', 'https://medlineplus.gov/spanish/recetas/sopas/', 'https://medlineplus.gov/spanish/recetas/vegetariana/']


In [26]:
mat_enlaces = []

categorias = [
    "Desayuno",
    "Almuerzo",
    "Cena",
    "Postres",
    "Acompañamientos",
    "Bajo en grasa",
    "Bajo en sodio",
    "Bebidas",
    "Bocadillos",
    "Ensaladas",
    "Pan",
    "Salsas y aderezos",
    "Sin gluten",
    "Sin lácteos",
    "Sopas",
    "Vegetariana"
]

diccionario_enlaces = {categoria: [] for categoria in categorias}

i = 0

for enlace in enlaces_secciones:

    response = requests.get(enlace)
    soup = BeautifulSoup(response.content, "html.parser")

    div_cards = soup.find_all('div', class_='recipe-card-block-with-height')


    for card in div_cards:
        diccionario_enlaces[categorias[i]].append(card.find("a").get("href"))

    i = i + 1


In [27]:
# max_len = max(len(v) for v in diccionario_enlaces.values())

# for clave, lista in diccionario_enlaces.items():
#     if len(lista) < max_len:
#         diccionario_enlaces[clave] += [None] * (max_len - len(lista))

# df = pd.DataFrame(diccionario_enlaces)

# df.head(100)

In [30]:
 # "Categoría";"Enlace";"Titulo receta";"imagen";"Calorías";"Ingredientes";"Valores";"Vitaminas/Minerales";

datos = []

datos.append(["Categoría","Enlace","Nombre","ImagenURL","Calorías","Ingredientes","Nutrientes","Vitaminas y Minerales"])

for categoria, lista in diccionario_enlaces.items():
    print(categoria)

    for enlace in lista: # Reemplazar por categoría
        
        response = requests.get(enlace)

        ingredientes = ""
        factsM = ""
        factsN = ""

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # SCRAPPING NOMBRE
            
            nombre = soup.find('h1').text

            print(nombre)
            print("\n")

            # SCRAPPING IMAGEN

            imagen = soup.find('div', class_='mp-img').find("img").get("src")

            # SCRAPPING CALORÍAS
            
            kcal = soup.find_all('span', class_='fact-right')[1].find("strong").text
            
            # SCRAPPING INGREDIENTES

            lista_ingredientes = soup.find('ul', class_='bulletlist')

            if lista_ingredientes:
                lista_ingredientes = lista_ingredientes.find_all("li")

                for li in lista_ingredientes:
                    ingredientes = ingredientes + " | " + li.text.strip()
            else :
                ingredientes = ""

            # SCRAPPING FACTS MAIN

            facts_main = soup.find('div', class_='facts-main').find_all("span", class_="fact-left")

            for span in facts_main:
                factsM = factsM + " | " + span.text.strip()

            # SCRAPPING FACTS NUTRI

            facts_nutri = soup.find('div', class_='facts-nutri').find_all("span", class_="fact-left")

            for span in facts_nutri:
                factsN = factsN + " | " + span.text.strip()

        else:
            print(f"Error al acceder a la página: {response.status_code}")

        datos.append([categoria,enlace,nombre,imagen,kcal,ingredientes,factsM,factsN])
        
    os.system('cls')

with open("assets/data/datos.csv", "w", newline="", encoding="utf-8") as archivo_csv:
    escritor = csv.writer(archivo_csv, delimiter=";")

    # Escribir la cabecera (opcional)
    escritor.writerow(datos[0])

    # Escribir los datos
    escritor.writerows(datos[1:])

# "Categoría";"Enlace";"Titulo receta";"imagen";"Calorías";"Ingredientes";"Valores";"Vitaminas/Minerales";

Desayuno
Avena de la noche a la mañana


Avena horneada con bayas


Avena sabrosa


Banana split para el desayuno


Barras de avena y frambuesas


Barras de mantequilla de maní y cereal


Batido energético de piña verde


Burritos de desayuno


Cazuela de pan francés con fruta


Galletas de calabaza para el desayuno


Granola al sartén


Hojuelas de avena remojadas durante la noche para uno


Licuado de arándanos y tofu


Licuado de naranja


Manzana al horno con arándanos rojos


Muffin integral de arándanos azules


Muffins de camote y naranja


Muffins de quiche y verduras


Pan de plátano y avena


Pan de trigo integral con calabacita


Panqueques de calabaza


Panqueques de maíz


Panqueques esponjosos de cereza


Panqueques favoritos sin huevo


Parfait de arándanos


Pudín de plátano con tofu


Salsa de cualquier baya


Tofu revuelto


Tortilla de verduras en taza


Tortilla o "frittata" de huevo con vegetales


Tostada francesa con puré de manzana


Waffles panqueques de avena 